In [2]:
import azureml
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
! mkdir ./aml_config

In [ ]:
%%writefile ~/aml_config/config.json
{
    "subscription_id": "XXXXX-XXXXXX-XXXXX-XXXXX",
    "resource_group": "<リソースグループ>",
    "workspace_name": "<　ワークスペースの名前>"
}

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [4]:
from azureml.core import Experiment

## お試し

In [5]:

# Experimentの作成
# この時点でworkspace上のExperimentsにmyexpの項目ができる
exp = Experiment(workspace=ws, name='myexp')

# Runを作成，実行
# この時点でworkspace上のExperiments->myexp内にrun:1の項目ができる
run = exp.start_logging()

# Runにログを上げてみる
# これによりmyexp->run:1内のTracked Metricsに'my magic number': 42が付与される
run.log('my magic number', 42)

# listをあげる (Fibonacci numbers)
# グラフとなって表示される
run.log_list('my list', [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]) 

# 同じkeyのログを複数回上げてもグラフとなって表示される
run.log('next magic number', 42)
run.log('next magic number', 43)

# finish the run
# statusがcompleteとなる
run.complete()

## MNIST + Keras
* ハイパーパラメータを保存
* 実行結果のhistoryをグラフに保存
* 実行結果のモデルをファイルとして保存

In [17]:
from keras.datasets import mnist
import keras
from keras.callbacks import ModelCheckpoint

* データ準備

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784) 
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')   
x_test = x_test.astype('float32')
x_train /= 255                        
x_test /= 255

In [10]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

* ハイパーパラメータの設定

In [27]:
batch_size = 128
epochs = 10
lr = 0.01
optname = 'adam'

In [30]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [31]:
def choice_opt(opt, lr):
    if opt == 'sgd':
        return keras.optimizers.SGD(lr=lr)
    if opt == 'adam':
        return keras.optimizers.Adam(lr=lr)
    else:
        raise ValueError()

In [32]:
opt = choice_opt(optname, lr)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

In [33]:
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size,
    callbacks=[ModelCheckpoint('./best_model.h5', monitor='val_loss', save_best_only=True)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.2294 - acc: 0.9307 - val_loss: 0.1425 - val_acc: 0.9561
Epoch 2/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.1189 - acc: 0.9642 - val_loss: 0.1381 - val_acc: 0.9624
Epoch 3/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.1023 - acc: 0.9691 - val_loss: 0.1379 - val_acc: 0.9628
Epoch 4/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.0891 - acc: 0.9731 - val_loss: 0.1149 - val_acc: 0.9674
Epoch 5/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.0830 - acc: 0.9758 - val_loss: 0.1346 - val_acc: 0.9663
Epoch 6/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.0776 - acc: 0.9768 - val_loss: 0.1352 - val_acc: 0.9688
Epoch 7/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.0723 - acc: 0.9791 - val_loss: 0.1428 - val_acc

In [ ]:
exp = Experiment(workspace=ws, name='my_mnistexp')
run = exp.start_logging()

for k, v in history.history.items():
    run.log_list(k, v)

# ハイパーパラメータの記録
run.log('epochs', epochs)
run.log('batch_size', batch_size)
run.log('opt', optname)
run.log('lr', lr)

# モデルファイルのアップロード
# name: Azure上の保管場所, path_or_stream: ローカルのファイルパス
run.upload_file(name='outputs/bast_model.h5', path_or_stream = './best_model.h5') 

In [29]:
run.complete()

# Keras Callback

In [35]:
class AzureCallback(keras.callbacks.Callback):
    def __init__(self, run):
        super().__init__()
        self.run = run
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        train_loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_acc')
        self.run.log('loss', train_loss)
        self.run.log('val_loss', val_loss)
        self.run.log('val_acc', val_acc)
        
        

In [36]:
exp = Experiment(workspace=ws, name='my_mnistexp')
run = exp.start_logging()

In [37]:
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size,
    callbacks=[ModelCheckpoint('./best_model.h5', monitor='val_loss', save_best_only=True),
              AzureCallback(run)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0628 - acc: 0.9822 - val_loss: 0.1660 - val_acc: 0.9648
Epoch 2/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.0624 - acc: 0.9840 - val_loss: 0.1369 - val_acc: 0.9700
Epoch 3/10
60000/60000 [==============================] - 2s 33us/step - loss: 0.0635 - acc: 0.9834 - val_loss: 0.1459 - val_acc: 0.9707
Epoch 4/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.0577 - acc: 0.9850 - val_loss: 0.1374 - val_acc: 0.9718
Epoch 5/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.0497 - acc: 0.9867 - val_loss: 0.1637 - val_acc: 0.9725
Epoch 6/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.0508 - acc: 0.9875 - val_loss: 0.1649 - val_acc: 0.9704
Epoch 7/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.0589 - acc: 0.9860 - val_loss: 0.1824 - val_acc

In [38]:
run.complete()